In [1]:
import os
import copy
import json
import logging

import torch
from torch.utils.data import TensorDataset

In [2]:
if __name__ == "__main__":
    f=open('./ner_data/dev/input.txt', encoding='utf-8')
    line = f.readline().strip() #读取第一行
    sentence1 =[]
    sentence1.append(line)
    while line:  # 直到读取完文件
       line = f.readline().strip()  # 读取一行文件，包括换行符
       sentence1.append(line)
    f.close()  # 关闭文件
    
    f=open('./ner_data/dev/output.txt', encoding='utf-8')
    line = f.readline().strip() #读取第一行
    labels1 =[]
    labels1.append(line)
    while line:  # 直到读取完文件
       line = f.readline().strip()  # 读取一行文件，包括换行符
       labels1.append(line)
    f.close()  # 关闭文件
    


In [5]:
sentence1

['医 生 ： 你 好 我 是 您 的 接 诊 医 生',
 '医 生 ： 宝 贝 最 近 吃 奶 量 可 以 吗 ？ 下 降 了 吗',
 '患 者 ： 没 有 ， 也 没 怎 么 哭 闹',
 '医 生 ： 宝 妈 有 没 有 吃 生 冷 辛 辣 刺 激 食 物 油 腻 食 物 来 吗 ？',
 '医 生 ： 宝 贝 奶 粉 的 话 最 近 换 过 牌 子 吗 ？',
 '医 生 ： 宝 贝 肚 子 着 凉 来 吗 ？',
 '患 者 ： 喝 茶 油 腻 也 少 ， 菜 吃 很 多',
 '医 生 ： 嗯 嗯 ， 宝 妈 饮 食 一 定 注 意 ， 生 冷 辛 辣 刺 激 食 物 不 能 吃 油 腻 食 物 不 能 吃 ， 特 别 油 腻 食 物 的 奥 ， 清 淡 饮 食 为 主 ， 这 个 时 候 宝 贝 胃 肠 功 能 可 能 会 有 影 响 ， 能 吃 多 少 吃 多 少 别 强 喂 的 奥 ！',
 '医 生 ： 宝 贝 最 近 有 没 有 呕 吐 症 状 呢 ？',
 '患 者 ： 呕 吐 ， 有 时 会 吐 ， 不 多',
 '医 生 ： 嗯 嗯 ， 如 果 那 种 漾 奶 ， 每 次 量 很 少 的 话 就 问 题 不 大 ， 这 个 时 候 小 婴 儿 胃 处 于 水 平 位 ， 所 以 很 容 易 漾 奶 的 奥 ， 看 好 宝 贝 别 呛 到 了 就 可 以',
 '医 生 ： 宝 贝 如 果 呕 吐 量 非 常 大 的 话 就 要 给 宝 贝 空 肚 子 的 奥 ， 至 少 2 小 时 不 要 喝 水 不 要 吃 奶 的',
 '医 生 ： 宝 贝 拉 肚 子 到 今 天 为 止 总 共 第 几 天 了 呢',
 '患 者 ： 现 在 不 敢 喂 多 ， 但 吃 不 饱 宝 宝 又 会 闹 ， 就 9 0 毫 升 ， 加 点 水 这 样 。',
 '医 生 ： 嗯 嗯 ， 这 个 时 候 您 按 需 哺 乳 就 可 以 ， 能 喝 多 少 喝 多 少 别 强 喂 就 可 以 的',
 '医 生 ： 宝 贝 大 便 的 话 现 在 一 天 多 少 次 呢 ？',
 '患 者 ： 好 的 好 的',
 '医 生 ： 宝 贝 到 今 天 为 止 拉 肚 子 总 共 几 天 了 呢 ？',
 '患 者 ： 多 多 少 少 十 来 次',
 '医 

In [3]:
labels1

['O O O O O',
 'O O O O O O O O O O O',
 'O O O O O O O O O O O',
 'O O O O O',
 'O O O O O O O O O O',
 'O O O O O O O O O O O',
 'O O O O O O',
 'O O O O O O O O O O O',
 'O O O O O O O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O O O O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O O O B-Symptom I-Symptom O O',
 'O O O O O O B-Symptom I-Symptom I-Symptom',
 'O O O O O O B-Symptom I-Symptom O O O O',
 'O O O O O O O O O',
 'O O O O O O B-Symptom I-Symptom O O O O',
 'O O O B-Symptom I-Symptom O O O O O O O O O',
 'O O O O O',
 'O O O B-Symptom I-Symptom O O O O O O O O O O O O O O O O O O O O',
 'O O O O O O O O O O O O O O O O O O',
 'O O O O O O',
 'O O O O O O O O O B-Symptom I-Symptom O',
 'O O O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O B-Symptom I-Symptom O O O O',
 'O O O O O O O O B-Symptom O',
 'O O O O O O',
 'O O O O O O O O',
 'O O O O',
 'O O O O O O O O O O O O O O O',
 'O O O O O B-Symptom I-Symptom I-Symptom O',
 'O O O O O O',


In [44]:
sentence = []
for i in sentence1:
    d = sentence.extend(i.split())

In [45]:
sentence

['医',
 '生',
 '：',
 '你',
 '好',
 '医',
 '生',
 '：',
 '我',
 '是',
 '您',
 '的',
 '接',
 '诊',
 '医',
 '生',
 '医',
 '生',
 '：',
 '能',
 '收',
 '到',
 '回',
 '复',
 '吗',
 '？',
 '亲',
 '患',
 '者',
 '：',
 '收',
 '到',
 '医',
 '生',
 '：',
 '宝',
 '贝',
 '多',
 '大',
 '了',
 '？',
 '亲',
 '医',
 '生',
 '：',
 '最',
 '近',
 '吃',
 '饭',
 '怎',
 '么',
 '样',
 '呢',
 '患',
 '者',
 '：',
 '2',
 '岁',
 '半',
 '医',
 '生',
 '：',
 '吃',
 '饭',
 '有',
 '没',
 '有',
 '改',
 '变',
 '啊',
 '医',
 '生',
 '：',
 '有',
 '没',
 '有',
 '吃',
 '什',
 '么',
 '不',
 '合',
 '适',
 '东',
 '西',
 '呢',
 '患',
 '者',
 '：',
 '中',
 '午',
 '吃',
 '的',
 '面',
 '条',
 '医',
 '生',
 '：',
 '有',
 '没',
 '有',
 '吃',
 '生',
 '冷',
 '辛',
 '辣',
 '刺',
 '激',
 '食',
 '物',
 '油',
 '腻',
 '食',
 '物',
 '呢',
 '患',
 '者',
 '：',
 '还',
 '有',
 '一',
 '个',
 '鸡',
 '蛋',
 '医',
 '生',
 '：',
 '嗯',
 '啊',
 '，',
 '宝',
 '贝',
 '主',
 '要',
 '是',
 '呕',
 '吐',
 '对',
 '吗',
 '患',
 '者',
 '：',
 '有',
 '一',
 '点',
 '拉',
 '肚',
 '子',
 '医',
 '生',
 '：',
 '嗯',
 '啊',
 '，',
 '呕',
 '吐',
 '几',
 '天',
 '了',
 '呢',
 '患',
 '者',
 '：',
 '下',
 '午',
 '5',
 '点',
 '左'

In [46]:
labels = []
for i in labels1:
    d = labels.extend(i.split())

In [ ]:
# 数据格式转变
filename = './ner_data/dev/test.txt'
with open(filename,'w',encoding='utf-8') as f:
    for i in range(len(sentence)):
        f.write(sentence[i]+ ' '+ labels[i] +'\n')
        if i < len(sentence)-3 and i>1:
            if sentence[i+1]=='医' and sentence[i+2]=='生' and sentence[i+3]=='：':
                f.write('\n')
                
            if sentence[i+1]=='患' and sentence[i+2]=='者' and sentence[i+3]=='：':
                f.write('\n')

In [5]:
X = []
y = []
X.append(sentence.copy())
y.append(labels.copy())

In [1]:
import re
import pandas as pd
from bert4keras.snippets import sequence_padding,DataGenerator
from bert4keras.tokenizers import Tokenizer

In [7]:
entity_labels = ['Symptom','Operation','Medical_Examination','Drug_Category','Drug']

id2label = {i:j for i,j in enumerate(sorted(entity_labels))}
label2id = {j:i for i,j in id2label.items()}

num_labels = len(entity_labels) * 2 + 1

vocab_path = './bert_weight_file/uncased_L-4_H-768_A-12/vocab.txt'
tokenizer = Tokenizer(vocab_path,do_lower_case=True)

datasets = []
samples_len = []

In [8]:
for token_seq,label_seq in zip(X,y):
    if len(token_seq) < 2:
        continue
    sample_seq, last_flag = [], ''
    for token, this_flag in zip(token_seq,label_seq):
        if this_flag == 'O' and last_flag == 'O':
            sample_seq[-1][0] += token
        elif this_flag == 'O' and last_flag != 'O':
            sample_seq.append([token, 'O'])
        elif this_flag[:1] == 'B':
            sample_seq.append([token, this_flag[2:]])
        else:
            sample_seq[-1][0] += token
        last_flag = this_flag
    datasets.append(sample_seq)
    samples_len.append(len(token_seq))

In [2]:
class data_generator(DataGenerator):  # 数据生成器
    def __iter__(self, random=True):
        batch_token_ids,batch_segment_ids,batch_labels = [], [], []
        for is_end,item in self.sample(random):
            token_ids, labels = [tokenizer._token_start_id], [0] #[CLS]
            for w, l in item:
                w_token_ids = tokenizer.encode(w)[0][1:-1]
                if len(token_ids) + len(w_token_ids) < 100:
                    token_ids += w_token_ids
                    if l =='O':
                        labels += [0] * len(w_token_ids)
                    else:
                        B = label2id[l] * 2 +1
                        I = label2id[l] * 2 +1
                        labels += ([B] + [I] * (len(w_token_ids) - 1)) 
                else:
                    break
            token_ids += [tokenizer._token_end_id] # [seq]
            labels += [0]
            segment_ids = [0] * len(token_ids)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append(labels)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []
                
def load_data(data_path,max_len):
    
    sentence = []
    labels = []
    X = []
    y = []
    datasets = []
    samples_len = []
    split_pattern = re.compile(r'[;；。，、？\.\?!]')
    with open(data_path,'r',encoding= 'utf8') as f:
        for line in f.readlines():
            line = line.strip().split()
            if(not line or len(line) < 2):
                X.append(sentence.copy())
                y.append(labels.copy())
                sentence.clear()
                labels.clear()
                continue
            word, tag = line[0], line[1]
            if split_pattern.match(word) and len(sentence) >= max_len:
                sentence.append(word)
                labels.append(tag)
                sentence.clear()
                labels.clear()
            else:
                sentence.append(word)
                labels.append(tag)
    if len(sentence):
        X.append(sentence.copy())
        sentence.clear()
        y.append(labels.copy())
        labels.clear()

    for token_seq,label_seq in zip(X,y):
        if len(token_seq) < 2:
            continue
        sample_seq, last_flag = [], ''
        for token, this_flag in zip(token_seq,label_seq):
            
            if this_flag == 'O' and last_flag == 'O':
                sample_seq[-1][0] += token
            elif this_flag == 'O' and last_flag != 'O':
                sample_seq.append([token, 'O'])
            elif this_flag[:1] == 'B':
                sample_seq.append([token, this_flag[2:]]) 
                save = token
            elif this_flag[:1] == 'I' and last_flag[:1] == 'B':
                del sample_seq[-1][-1] 
                del sample_seq[-1][-1] 
                sample_seq.append([save+token, this_flag[2:]])
                save = save+token
            elif this_flag[:1] == 'I' and last_flag[:1] == 'I':
                del sample_seq[-1][-1] 
                del sample_seq[-1][-1]
                sample_seq.append([save+token, this_flag[2:]])
                save = save+token
            last_flag = this_flag
        datasets.append([x for x in sample_seq if x != []])       
        samples_len.append(len(token_seq))
    
    df = pd.DataFrame(samples_len)
    print(data_path,'\n',df.describe())
    return datasets,y


In [3]:
if __name__ == '__main__':
    data_path = './ner_data/train/train.txt'
    # data_path = 'E:/工作空间/bert_for_ner/data/train.conll'
    d,y= load_data(data_path,128)
    print(d[:3])

./ner_data/train/train.txt 
                   0
count  59475.000000
mean      15.839731
std       13.389132
min        2.000000
25%        8.000000
50%       12.000000
75%       19.000000
max      157.000000
[[['医生：你好我是您的接诊医生', 'O']], [['医生：宝贝最近吃奶量可以吗？下降了吗', 'O']], [['患者：没有，也没怎么', 'O'], ['哭闹', 'Symptom']]]
